In [ ]:
import os
import nltk
import numpy as np
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
assets_dir = os.path.realpath("../assets/annotated-corpus")
train_dir = os.path.join(assets_dir, "test")

In [ ]:
topics = os.listdir(train_dir)

In [ ]:
sentences = []
for t in topics:
    workdir = os.path.join(train_dir, t)
    for filename in os.listdir(workdir):
        with open(os.path.join(workdir, filename)) as f:
            lines = "".join(f.readlines())
            sentences_raw = lines.split("\n\n")
            for s in sentences_raw:
                words = s.split("\n")
                if len(words) == 0 or words[0] == "":
                    continue
                stems_raw = list(map(lambda x: x.split("\t")[1], words))
                lemmas = list(map(lambda x: x.split("\t")[2], words))
                stems = []
                for i in range(len(stems_raw)):
                    if lemmas[i] not in stopwords.words("english"):
                        stems.append(stems_raw[i])
                sentences.append(stems)


In [ ]:
len(sentences)

In [ ]:
ngram_length = 3

In [ ]:
ngrams = []
word_count = {}
ngrams_count = {}
for s in sentences:
    counter = 0
    for w in s:
        if w not in word_count.keys():
            word_count[w] = 0
        word_count[w] += 1
        counter += 1
    if counter < ngram_length:
        continue
    for i in range(len(s) - ngram_length + 1):
        ngram = tuple(s[i:i+ngram_length])
        if ngram not in ngrams_count.keys():
            ngrams_count[ngram] = 0
        ngrams_count[ngram] += 1
        ngrams.append(ngram)


In [ ]:
len(ngrams)

In [ ]:
sorted(word_count.items(), key=lambda x: -x[1])[:30]

In [ ]:
sorted(ngrams_count.items(), key=lambda x: -x[1])[:30]

In [ ]:
total_words = sum(word_count.values())
total_words


### f(n, c) - ngrams_count[ngram], частота встречаемости ключевого слова n в паре с коллокатом c;
### N - total_words, общее число словоупотреблений в корпусе (тексте);
### П_i f(u_i) - count_mul_result, Произведение абсолютных частот i-й униграммы в 3-грамме

In [ ]:
ngram_score = {}
for ngram in set(ngrams):
    count_mul_result = 1
    for word in ngram:
        count_mul_result *= word_count[word]
    ngram_score[ngram] = np.log2(ngrams_count[ngram] * (total_words**(ngram_length-1)) / count_mul_result)
sorted(ngram_score.items(), key=lambda x: -x[1])[0:30]


In [ ]:
ngram_score = {}
for ngram in set(ngrams):
    count_sum_log_result = 0
    for word in ngram:
        count_sum_log_result += np.log2(word_count[word])
    ngram_score[ngram] = np.log2(ngrams_count[ngram]) + (ngram_length-1)*np.log2(total_words) - count_sum_log_result
sorted(ngram_score.items(), key=lambda x: -x[1])[30:60]

In [ ]:
from nltk.collocations import  *
from nltk import Text
import nltk
text = []
for s in sentences:
    text += s
finder = TrigramCollocationFinder.from_words(Text(text))
finder.nbest(nltk.collocations.TrigramAssocMeasures().mi_like, 30)